In [ ]:
if __package__ is None:
    import sys
    from os import path
    sys.path.append(path.dirname(path.dirname(path.abspath("Test.ipynb"))))

from FMLC.triggering import triggering
from FMLC.baseclasses import eFMU
from FMLC.stackedclasses import controller_stack
import time
from datetime import datetime
import pandas as pd


In [ ]:
class testcontroller1(eFMU):
    def __init__(self):
        self.input = {'a': None, 'b': None}
        self.output = {'c': None}
        self.init = True
    def compute(self):
        self.init= False
        self.output['c'] = self.input['a'] * self.input['b']


class testcontroller2(eFMU):
    def __init__(self):
        self.input = {'a': None, 'b': None}
        self.output = {'c': None}
        self.init = True

    def compute(self):
        self.init = False
        self.output['c'] = self.input['a'] * self.input['b']
        time.sleep(0.2)


class testcontroller3(eFMU):
    def __init__(self):
        self.input = {'a': None, 'b': None}
        self.output = {'c': None}
        self.init = True

    def compute(self):
        self.init = False
        self.output['c'] = self.input['a'] * self.input['b']
        time.sleep(1)

class testcontroller4(eFMU):
    def __init__(self):
        self.input = {'a': None, 'b': None}
        self.output = {'c': None}
        self.init = True

    def compute(self):
        self.init = False
        self.output['c'] = self.input['a'] * self.input['b']
        time.sleep(10)
controller = {}
controller['forecast1'] = {'fun':testcontroller1, 'sampletime':0}
controller['mpc1'] = {'fun':testcontroller3, 'sampletime':'forecast1'}
controller['control1'] = {'fun':testcontroller1, 'sampletime':'mpc1'}
controller['forecast2'] = {'fun':testcontroller3, 'sampletime':0}
controller['forecast3'] = {'fun':testcontroller1, 'sampletime':0}
controller = controller_stack(controller, tz=-8, debug=True, parallel=True, timeout=0.8)

mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'
controller.initialize(mapping)

for i in range(6):
    controller.query_control(time.time())
    time.sleep(1.5)

for df in controller.log_to_df().values():
    display(df)

In [ ]:
controller.query_control(time.time())
for df in controller.log_to_df().values():
    display(df)

In [ ]:
display(controller.running_controllers.__str__())
display(controller.executed_controllers.__str__())
display(controller.timeout_controllers.__str__())

In [ ]:
display(len(controller.log_to_df()['mpc1'].columns))
display(controller.running_controllers.__str__())
display(controller.executed_controllers.__str__())
display(controller.timeout_controllers.__str__())

In [ ]:
for i in range(3):
    controller.query_control(time.time())
    time.sleep(1)
for df in controller.log_to_df().values():
    display(df)

In [ ]:
display(len(controller.log_to_df()['mpc1'].columns))
display(controller.running_controllers.__str__())
display(controller.executed_controllers.__str__())
display(controller.timeout_controllers.__str__())